In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Importiere MINST Daten
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#mint=tf.keras.datasets.mnist
#(x_,y_),(x_1,y_1)=mint.load_data()

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
#print ("Download and Extract MNIST dataset")
mnist = input_data.read_data_sets('data/', one_hot=True)

In [ ]:
from functools import wraps
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

## Daten Format

Die Daten sind im Vektor Format gespeichert, obwohl die Originaldaten eine 2-dimensionale Matrix waren, die angab, wie viele Pigmente sich an welcher Position befinden. Untersuchen wir das genauer:

In [ ]:
type(mnist)

In [ ]:
type(mnist.train.images)

In [ ]:
type(mnist.test.images)

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
#print("Fit model on training data")
#history = model.fit(
#    x_train,
#    y_train,
#    batch_size=64,
#    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
#    validation_data=(x_val, y_val),
#)

In [ ]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        #logging.info('Ran with args: {}'.format(args))
        logging.info(orig_func(*args))
        return orig_func(*args)

    return wrapper

def my_timer(orig_func):
    import time
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        t1 = time.time()
        result = orig_func(*args)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        logging.info('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [157]:
@my_logger
@my_timer
def fit(x_train, y_train): 
    return model.fit(x_train, y_train)
    
@my_logger
@my_timer
def predict(x_test):
    return model.predict(x_test)

In [159]:
print(fit(x_train, y_train))

print(predict(x_test))

ValueError: Input arrays should have the same number of samples as target arrays. Found 55000 input samples and 10000 target samples.

In [ ]:
#mnist.train.images[0]
mnist.train.images[2].shape

In [ ]:
sample = mnist.train.images[2].reshape(28,28)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(sample)

## Parameter

Wir werden 4 Parameter definieren müssen. Es ist wirklich (wirklich) schwer gute Parameterwerte für einen Datensatz zu bestimmen, mit dem man keine Erfahrung hat. Da dieser MNIST Datensatz allerdings so berühmt ist haben wir schon einige Ausgangswerte. Die Parameter sind:

* Learning Rate - Wie schnell die Kostenfunktion angepasst wird
* Traing Epochs - Wie viele Trainingszyklen durchlaufen werden sollen
* Batch Size - Größe der "Batches" an Traingsdaten

In [ ]:
# Parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100

### Netzwerk Parameter

Hier haben wir Parameter welche unser Neuronales Netz direkt definieren. Diese werden entsprechend der betrachteten Daten angepasst und hängen auch davon ab, welche Art von Netz man nutzt. Es sind bis zu diesem Punkt erst einmal nur Zahlen, die wir später verwenden, um unser Netz zu definieren:

In [ ]:
# Netzwerk Parameter
n_hidden_1 = 256 # 1. Layer: Anzahl an Features
n_hidden_2 = 256 # 2. Layer: Anzahl an Features
n_input = 784 # MNIST Daten Input (img shape: 28*28)
n_classes = 10 # MNIST Klassen (0-9 Zahlen)
n_samples = mnist.train.num_examples

### Tensorflow Graph Input

In [ ]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

## MultiLayer Modell

Es ist Zeit unser Modell zu erstellen. Wiederholen wir deshalb kurz, was wir erstellen wollen:

Zuerst erhalten wir einen *Input* in Form eines Datenarrays und schicken diesen an die erste *Hidden Layer*. Dann wird den Daten ein  *Weight* zwischen den Schichten zugewiesen (welches zuerst ein zufälliger Wert ist). Anschließend wird es an einen *Node* geschicht und unterläuft eine *Activation Function* (zusammen mit einem Bias, wie in der Neural Network Lektion erwähnt). Dann geht es weiter zur nächsten *Layer* und immer so weiter, bis zur finalen *Output Layer*. In unserem Fall werden wir nur 2 *Hidden Layers* verwenden. Je mehr wir davon verwenden, desto länger braucht das Modell (aber er hat mehr Möglichkeiten um die Genauigkeit zu erhöhen).

Sobald die transformierte Daten die *Output Layer* erreicht haben müssen wir sie auswerten. Hier verwenden wir eine *Loss Function* (auch Cost Function genannt). Diese berechnet, wie sehr wir vom gewünschten Ergebnis entfernt sind. In diesem Fall: Wie viele der Klassen wir richtig zugeteilt haben.

Dann wenden wir eine Optimierungsfunktion an, um die *Costs* (bzw. den Error) zu minimieren. Dies geschiet durch die Anpassung der *Weights* entlang des Netzes. Wir verwenden in unserem Beispiel den [Adam Optimizer](https://arxiv.org/pdf/1412.6980v8.pdf), welcher eine (im Vergleich zu anderen) sehr neue Entwicklung ist.

Wir können anpassen, wie schnell diese Optimierung angewendet wird, indem wir unseren *Learning Rate* Parameter anpassen. Je geringer die Rate, desto höher die Möglichkeiten für Anpassungen. Dies erzeugt allerdings die Kosten einer erhöhten Wartezeit. Ab einem bestimmten Punkt lohnt es sich nicht mehr, die Learning Rate weiter zu senken.

Jetzt können wir unser Modell erstellen. Wir beginnen mit 2 Hidden Layers, welche die []() Activation Function verwenden. Dies ist eine einfache Umformungsfunktion, die entweder x oder 0 zurückgibt. Für unsere finale Output Layer verwenden wir eine lineare Activation mit Matrixmultiplikation:

In [ ]:
def multilayer_perceptron(x, weights, biases):
    '''
    x : Platzhalter für den Dateninput
    weights: Dictionary der Weights
    biases: Dictionary Der Biases
    '''
    
    # Erste Hidden layer mit RELU Activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Zweite Hidden layer mit RELU Activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Letzte Output layer mit linearer Activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

## Weights und Bias

Damit unser Tensorflow Modell funktioniert müssen wir zwei Dictionaries anlegen, die unsere Weights und Biases enthalten. Wir können das `tf.variable` Objekt verwenden. Dies ist anders als eine Konstante, da Tensorflow's Graph Objekt alle Zustände der Variablen wahrnimmt. Eine Variable ist ein anpassbares Tensor, der zwischen Tensorflow's Graph von interagierenden Operationen lebt. Er kann durch die Berechnung verwendet und verändert werden. Wir werden die Modell Parameter generell als Variablen verwenden. Aus der Dokumentation können wir entnehmen:

    A variable maintains state in the graph across calls to `run()`. You add a variable to the graph by constructing an instance of the class `Variable`.

    The `Variable()` constructor requires an initial value for the variable, which can be a `Tensor` of any type and shape. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.
    
Wir werden Tensorflow's eingebaute `random_normal` Methode verwenden, um zufällige Werte für unsere Weights und Biases zu erstellen.

In [ ]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

In [ ]:
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# Model erstellen
pred = multilayer_perceptron(x, weights, biases)

## Cost und Optimierungs-Funktion

Wir verwenden Tensorflow's eingebaute Funktionen für diesesn Teil. Weitere Details bietet die Dokumentation:

In [ ]:
# Cost und Optimierungsfunktion definieren
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits=pred))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Initialisierung der Variablen

Wir initialisieren nun alle tf.Variable Objekte die wir zuvor erstellt haben. Das wird das erste sein, dass wir ausführen, wenn wir unser Modell trainieren.

In [ ]:
init = tf.global_variables_initializer()

## Das Modell trainieren

### next_batch()

Bevor wir beginnen möchte ich eine weitere nützliche Funktion in unserem MNIST Datenobjekt abdecken, die `next_batch` heißt. Diese gibt ein Tupel in der Form (X,y) mit einem X Array der Daten und einem y Array der Klasse. Zum Beispiel:

In [ ]:
Xsamp,ysamp = mnist.train.next_batch(1)

In [ ]:
plt.imshow(Xsamp.reshape(28,28))

In [ ]:
print(ysamp)

## Die Session ausführen

Jetzt ist es Zeit unsere Session auszuführen! Achte darauf wie wir zwei Schleifen verwenden. Die äußere, die die Epochs durchläuft, und die innere, die die Batches für jede Epoch des Trainings ausführt.

In [130]:
# starte Session
sess = tf.InteractiveSession()

# Variablen initialisieren
sess.run(init)

# Training Epochs
for epoch in range(training_epochs):

    # Start mit cost = 0.0
    avg_cost = 0.0

    # Konvertiere die Anzahl an Batches in eine Integer
    total_batch = int(n_samples/batch_size)

    # Schleife für alle Batches
    for i in range(total_batch):

        # Den nächsten Batch an Trainingsdaten und -lables nehmen
        batch_x, batch_y = mnist.train.next_batch(batch_size)

        # Dictionary für Optimierung und Cost bereitstellen
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})

        # Durchschnittliche Kosten berechnen
        avg_cost += c / total_batch

    print("Epoch: {} Cost={:.4f}".format(epoch+1,avg_cost))

print("Modellierung ist beendet: {} Epochs of Training".format(training_epochs))

Epoch: 1 Cost=144.9854
Epoch: 2 Cost=35.4830
Epoch: 3 Cost=22.2157
Epoch: 4 Cost=15.5948
Epoch: 5 Cost=11.2357
Epoch: 6 Cost=8.1410
Epoch: 7 Cost=6.1241
Epoch: 8 Cost=4.6070
Epoch: 9 Cost=3.2993
Epoch: 10 Cost=2.4559
Epoch: 11 Cost=1.9747
Epoch: 12 Cost=1.4505
Epoch: 13 Cost=0.9795
Epoch: 14 Cost=0.8657
Epoch: 15 Cost=0.7392
Modellierung ist beendet: 15 Epochs of Training


## Modell Auswertung

Tensorflow bietet einige eingebaute Funktionen, die uns bei der Auswertung helfen. Dazu gehören `tf.equal` und `tf.reduce_mean`.


### tf.equal

Dies ist im Grunde genommen nur eine Kontrolle, ob die Vorhersagen mit den Labels übereinstimmen. Da wir in unserem Fall wissen, dass die Labels eine 1 in einem Array von Nullen sind, können wir `argmax()` verwenden, um die Position zu vergleichen. Denke daran, dass y immer noch der Platzhalter ist, den wir anfangs erstellt haben. Wir werden eine Reihe an Operationen durchführen, um einen Tensor zu erhalten, in den wir die Testdaten einlesen können, um es auszuwerten.

In [134]:
# Teste das Modell
correct_predictions = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

test_accuracy = model.predict(x_test)
print()
print('Test Accuracy:', test_accuracy,'\n')


Test Accuracy: [[0.11472108 0.09253019 0.0974436  ... 0.09433733 0.08372135 0.13716567]
 [0.09443238 0.09526553 0.09923879 ... 0.13517469 0.07263706 0.11085503]
 [0.09343551 0.13342266 0.08519286 ... 0.11716598 0.08490753 0.11161031]
 ...
 [0.09795309 0.06142256 0.09964161 ... 0.1101485  0.08453584 0.15440579]
 [0.08523151 0.1007003  0.08450088 ... 0.13983455 0.07054286 0.12634292]
 [0.07983106 0.11712478 0.09230536 ... 0.17757612 0.08440571 0.0935029 ]] 



In [135]:
print(correct_predictions[0])

Tensor("strided_slice:0", shape=(), dtype=bool)


Um numerische Werte für unsere Vorhersagen zu erhalten müssen wir `tf.cast` verwenden, um den Tensor mit Booleans zurückzuführen in einen Tensor mit Floats. Dann können wir den Durchschnitt nehmen.

In [136]:
correct_predictions = tf.cast(correct_predictions, "float")

In [137]:
print(correct_predictions[0])

Tensor("strided_slice_1:0", shape=(), dtype=float32)


Jetzt können wir `tf.reduce_mean` verwenden, um den Durchschnitt der Elemente im Tensor zu erhalten:

In [138]:
accuracy = tf.reduce_mean(correct_predictions)

In [139]:
type(accuracy)

tensorflow.python.framework.ops.Tensor

Das wirkt evtl. etwas merkwürdig, aber diese Genauigkeit ist immer noch ein Tensor Objekt. Denke daran, dass wir immer noch die tatsächlichen Testdaten übergeben müssen. Jetzt können wir die MNIST Testlabels und Bilder aufrufen und die Genauigkeit auswerten!

In [140]:
mnist.test.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [141]:
mnist.test.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Die `eval()` Methode erlaubt es uns direkt in der Session den Tensor auszuwerten ohne `tf.sess():mm` aufrufen zu müssen

In [142]:
print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Accuracy: 0.9433


In [149]:
x = mnist.train.images
y = mnist.test.images

In [150]:
@my_logger
@my_timer
def split(x,y, splitRatio):
    x_train = x[:splitRatio]
    y_train = y[:splitRatio]
    x_test = x[splitRatio:]
    y_test = y[splitRatio:]
    return (x_train, y_train, x_test, y_test)

In [154]:
splitRatio = 60000
x_train, y_train, x_test, y_test = split(x,y,splitRatio) 

np.random.seed(31337)
ta = TheAlgorithm(x_train, y_train, x_test, y_test)
train_accuracy = model.fit()
print()
print('Train Accuracy:', train_accuracy,'\n') 
print("Train confusion matrix:\n%s\n" % ta.train_confusion_matrix)
  
test_accuracy = model.predict()
print()
print('Test Accuracy:', test_accuracy,'\n') 
print("Test confusion matrix:\n%s\n" % ta.test_confusion_matrix)

split ran in: 6.9141387939453125e-06 sec
split ran in: 4.76837158203125e-06 sec


TypeError: TheAlgorithm() takes no arguments

94%! Nicht schlecht! Aber das ist noch lange nicht zu gut, wie wir es schaffen können. Mehr Trainingsdurchläuft (Epochs) mit diesen Daten (ca. 20000) können eine Genauigkeit von 99% erreichen. Da dies sehr lange dauert werden wir es hier nicht durchführen.

# Gut gemacht!